<a href="https://colab.research.google.com/github/pkhlingam09/Fashion-Search-AI/blob/EDA/Fashion_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Dataset Link (Kaggle)
# https://www.kaggle.com/datasets/djagatiya/myntra-fashion-product-dataset

In [ ]:
# !pip install -q kaggle

# from google.colab import files
# files.upload()

# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

# !cat ~/.kaggle/kaggle.json

# !kaggle datasets download "djagatiya/myntra-fashion-product-dataset" -p "/content/drive/MyDrive/Fashion_Search_AI"

# !unzip "/content/drive/MyDrive/Fashion_Search_AI/myntra-fashion-product-dataset.zip" -d "/content/drive/MyDrive/Fashion_Search_AI"

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
import ast
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("/content/drive/MyDrive/Fashion_Search_AI/Fashion Dataset v2.csv")

In [4]:
df.head()

,p_id,name,products,price,colour,brand,img,ratingCount,avg_rating,description,p_attributes
0,17048614,Khushal K Women Black Ethnic Motifs Printed Ku...,"Kurta, Palazzos, Dupatta",5099.0,Black,Khushal K,http://assets.myntassets.com/assets/images/170...,4522.0,4.418399,Black printed Kurta with Palazzos with dupatta...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
1,16524740,InWeave Women Orange Solid Kurta with Palazzos...,"Kurta, Palazzos, Floral Print Dupatta",5899.0,Orange,InWeave,http://assets.myntassets.com/assets/images/165...,1081.0,4.119334,Orange solid Kurta with Palazzos with dupatta<...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
2,16331376,Anubhutee Women Navy Blue Ethnic Motifs Embroi...,"Kurta, Trousers, Dupatta",4899.0,Navy Blue,Anubhutee,http://assets.myntassets.com/assets/images/163...,1752.0,4.161530,Navy blue embroidered Kurta with Trousers with...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
3,14709966,Nayo Women Red Floral Printed Kurta With Trous...,"Kurta, Trouser, Dupatta",3699.0,Red,Nayo,http://assets.myntassets.com/assets/images/147...,4113.0,4.088986,Red printed kurta with trouser and dupatta<br>...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
4,11056154,AHIKA Women Black & Green Printed Straight Kurta,Kurta,1350.0,Black,AHIKA,http://assets.myntassets.com/assets/images/110...,21274.0,3.978377,"Black and green printed straight kurta, has a ...","{'Body Shape ID': '424', 'Body or Garment Size..."


In [5]:
df.shape

(14214, 11)

In [6]:
df.isna().sum()

,0
p_id,0
name,0
products,0
price,0
colour,0
brand,0
img,0
ratingCount,7684
avg_rating,7684
description,0


In [7]:
round(100 * df.isna().sum()/df.shape[0], 3)

,0
p_id,0.000
name,0.000
products,0.000
price,0.000
colour,0.000
brand,0.000
img,0.000
ratingCount,54.059
avg_rating,54.059
description,0.000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14214 entries, 0 to 14213
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   p_id          14214 non-null  int64  
 1   name          14214 non-null  object 
 2   products      14214 non-null  object 
 3   price         14214 non-null  float64
 4   colour        14214 non-null  object 
 5   brand         14214 non-null  object 
 6   img           14214 non-null  object 
 7   ratingCount   6530 non-null   float64
 8   avg_rating    6530 non-null   float64
 9   description   14214 non-null  object 
 10  p_attributes  14214 non-null  object 
dtypes: float64(3), int64(1), object(7)
memory usage: 1.2+ MB


In [9]:
df.drop(["ratingCount", "avg_rating"], axis=1, inplace=True)

In [10]:
df.shape

(14214, 9)

In [12]:
df.head()

,p_id,name,products,price,colour,brand,img,description,p_attributes
0,17048614,Khushal K Women Black Ethnic Motifs Printed Ku...,"Kurta, Palazzos, Dupatta",5099.0,Black,Khushal K,http://assets.myntassets.com/assets/images/170...,Black printed Kurta with Palazzos with dupatta...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
1,16524740,InWeave Women Orange Solid Kurta with Palazzos...,"Kurta, Palazzos, Floral Print Dupatta",5899.0,Orange,InWeave,http://assets.myntassets.com/assets/images/165...,Orange solid Kurta with Palazzos with dupatta<...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
2,16331376,Anubhutee Women Navy Blue Ethnic Motifs Embroi...,"Kurta, Trousers, Dupatta",4899.0,Navy Blue,Anubhutee,http://assets.myntassets.com/assets/images/163...,Navy blue embroidered Kurta with Trousers with...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
3,14709966,Nayo Women Red Floral Printed Kurta With Trous...,"Kurta, Trouser, Dupatta",3699.0,Red,Nayo,http://assets.myntassets.com/assets/images/147...,Red printed kurta with trouser and dupatta<br>...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
4,11056154,AHIKA Women Black & Green Printed Straight Kurta,Kurta,1350.0,Black,AHIKA,http://assets.myntassets.com/assets/images/110...,"Black and green printed straight kurta, has a ...","{'Body Shape ID': '424', 'Body or Garment Size..."


In [18]:
len(df['p_id'].unique()), type(df['p_id'][0])

(14214, numpy.int64)

In [20]:
len(df["name"].unique())

13873

In [37]:
df["p_attributes"][0]

"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,324,424', 'Body or Garment Size': 'Garment Measurements in', 'Bottom Closure': 'Slip-On', 'Bottom Fabric': 'Viscose Rayon', 'Bottom Pattern': 'Printed', 'Bottom Type': 'Palazzos', 'Character': 'NA', 'Dupatta': 'With Dupatta', 'Dupatta Border': 'Solid', 'Dupatta Fabric': 'Viscose Rayon', 'Dupatta Pattern': 'Printed', 'Main Trend': 'Indie Prints', 'Neck': 'Mandarin Collar', 'Number of Pockets': 'NA', 'Occasion': 'Festive', 'Ornamentation': 'NA', 'Pattern Coverage': 'Placement', 'Print or Pattern Type': 'Ethnic Motifs', 'Sleeve Length': 'Three-Quarter Sleeves', 'Sleeve Styling': 'Regular Sleeves', 'Slit Detail': 'NA', 'Stitch': 'Ready to Wear', 'Sustainable': 'Regular', 'Technique': 'Screen', 'Top Design Styling': 'Regular', 'Top Fabric': 'Viscose Rayon', 'Top Hemline': 'Flared', 'Top Length': 'Calf Length', 'Top Pattern': 'Printed', 'Top Shape': 'Anarkali', 'Top Type': 'Kurta', 'Waistband': 'Elasticated', 'Wash Care': 'Machine Wash', 'Weave Pa

In [40]:
df["description"][0]

"Black printed Kurta with Palazzos with dupatta <br> <br> <b> Kurta design:  </b> <ul> <li> Ethnic motifs printed </li> <li> Anarkali shape </li> <li> Regular style </li> <li> Mandarin collar,  three-quarter regular sleeves </li> <li> Calf length with flared hem </li> <li> Viscose rayon machine weave fabric </li> </ul> <br> <b> Palazzos design:  </b> <ul> <li> Printed Palazzos </li> <li> Elasticated waistband </li> <li> Slip-on closure </li> </ul>Dupatta Length 2.43 meters Width:&nbsp;88 cm<br>The model (height 5'8) is wearing a size S100% Rayon<br>Machine wash"

In [35]:
df["p_attributes"][12]

"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', 'Body or Garment Size': 'Garment Measurements in', 'Bottom Closure': 'Zip', 'Bottom Fabric': 'Poly Silk', 'Bottom Pattern': 'Solid', 'Bottom Type': 'Trousers', 'Character': 'NA', 'Dupatta': 'With Dupatta', 'Dupatta Border': 'Taping', 'Dupatta Fabric': 'Art Silk', 'Dupatta Pattern': 'Printed', 'Main Trend': 'NA', 'Neck': 'Keyhole Neck', 'Number of Pockets': 'NA', 'Occasion': 'Festive', 'Ornamentation': 'NA', 'Pattern Coverage': 'Yoke or Border', 'Print or Pattern Type': 'Ethnic Motifs', 'Sleeve Length': 'Three-Quarter Sleeves', 'Sleeve Styling': 'Regular Sleeves', 'Slit Detail': 'Multiple Slits', 'Stitch': 'Ready to Wear', 'Sustainable': 'Regular', 'Technique': 'NA', 'Top Design Styling': 'Panelled', 'Top Fabric': 'Poly Silk', 'Top Hemline': 'Flared', 'Top Length': 'Calf Length', 'Top Pattern': 'Yoke Design', 'Top Shape': 'A-Line', 'Top Type': 'Kurta', 'Waistband': 'Partially Elasticated', 'Wash Care': 'Hand Wash', 'Weave Pattern': 'Regular

In [41]:
df["description"][12]

"Teal yoke design Kurta with Trousers with dupatta <br> <br> <b> Kurta design:  </b> <ul> <li> Ethnic motifs yoke design </li> <li> A-line shape </li> <li> Panelled style </li> <li> Keyhole neck,  three-quarter regular sleeves </li> <li> Calf length with flared hem </li> <li> Poly silk machine weave fabric </li> </ul> <br> <b> Trousers design:  </b> <ul> <li> Solid Trousers </li> <li> Partially elasticated waistband </li> <li> Zip closure </li> </ul>The model (height 5'8) is wearing a size S<br>Dupatta length 2.25 mtr Width 0.7 mtrKurta: Poly Silk<br>Bottom:&nbsp;Poly Silk<br>Dupatta: Art Silk<br>Hand wash"

In [34]:
df["p_attributes"][157]

"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', 'Body or Garment Size': 'Garment Measurements in', 'Bottom Closure': 'Slip-On', 'Bottom Fabric': 'Polyester', 'Bottom Pattern': 'Solid', 'Bottom Type': 'Trousers', 'Character': 'NA', 'Dupatta': 'With Dupatta', 'Dupatta Border': 'Tassels', 'Dupatta Fabric': 'Viscose Rayon', 'Dupatta Pattern': 'Printed', 'Main Trend': 'NA', 'Neck': 'Round Neck', 'Number of Pockets': 'NA', 'Occasion': 'Festive', 'Ornamentation': 'NA', 'Pattern Coverage': 'None', 'Print or Pattern Type': 'Woven Design', 'Sleeve Length': 'Three-Quarter Sleeves', 'Sleeve Styling': 'Regular Sleeves', 'Slit Detail': 'Side Slits', 'Stitch': 'Ready to Wear', 'Sustainable': 'Regular', 'Technique': 'NA', 'Top Design Styling': 'Regular', 'Top Fabric': 'Polyester', 'Top Hemline': 'Straight', 'Top Length': 'Calf Length', 'Top Pattern': 'Solid', 'Top Shape': 'Straight', 'Top Type': 'Kurta', 'Waistband': 'Elasticated', 'Wash Care': 'Machine Wash', 'Weave Pattern': 'Regular', 'Weave Type': 

In [39]:
df["description"][157]

"Purple solid Kurta with Trousers with dupatta <br> <br> <b> Kurta design:  </b> <ul> <li> Woven design solid </li> <li> Straight shape </li> <li> Regular style </li> <li> Round neck,  three-quarter regular sleeves </li> <li> Calf length with straight hem </li> <li> Polyester machine weave fabric </li> </ul> <br> <b> Trousers design:  </b> <ul> <li> Solid Trousers </li> <li> Elasticated waistband </li> <li> Slip-on closure </li> </ul>The model (height 5'8) is wearing a size SPolyester<br>Machine wash"

In [33]:
df["p_attributes"][9999]

"{'Blouse Closure': 'NA', 'Blouse Fabric': 'Silk', 'Body or Garment Size': 'Garment Measurements in', 'Bottom Pattern': 'Embroidered', 'Choli Stitch': 'Unstitched', 'Dupatta': 'With Dupatta', 'Dupatta Border': 'Tassels', 'Dupatta Fabric': 'Silk', 'Dupatta Pattern': 'Embellished', 'Hemline': 'Flared', 'Lehenga Fabric': 'Silk', 'Lehenga Lining Fabric': 'Satin', 'Lehenga Stitch': 'Semi-Stitched', 'Neck': 'Boat Neck', 'Ornamentation': 'Sequinned', 'Regions': 'All', 'Sleeve Length': 'Three-Quarter Sleeves', 'Sleeve Styling': 'Regular Sleeves', 'Sustainable': 'Regular', 'Technique': 'NA', 'Top Pattern': 'Embroidered', 'Wedding': 'Bride & Wedding Squad', 'Where-to-wear': 'Wedding &amp; Party'}"

In [38]:
df["description"][9999]

'White and maroon embroidered lehenga choli with dupatta,  sequinned <br> White and maroon embroidered unstitched blouse,  has a boat neck,  three-quarter sleeves,  white and maroon embroidered semi-stitched lehenga, flared hem <br> White and maroon embellished dupatta,  tassels border<p>Hand Wash</p><p>Lehenga Length: 44 Inch, Choli Length: 1 Mtr, Dupatta Length: 2.25 Mtr, Lehenga Gher: 3 Mtr</p>'

In [30]:
df["p_attributes"][12123]

"{'Add-Ons': 'NA', 'Body or Garment Size': 'Garment Measurements in', 'Character': 'NA', 'Closure': 'Zip', 'Collar': 'Stand Collar', 'Fabric': 'Nylon', 'Features': 'NA', 'Hemline': 'Straight', 'Length': 'Regular', 'Lining Fabric': 'Mesh', 'Main Trend': 'NA', 'Number of Pockets': '2', 'Occasion': 'Sports', 'Pattern': 'Printed', 'Print or Pattern Type': 'Geometric', 'Sleeve Length': 'Long Sleeves', 'Sport': 'NA', 'Surface Styling': 'NA', 'Sustainable': 'Regular', 'Technology': 'NA', 'Technology Present': 'No', 'Type': 'Sporty Jacket', 'Wash Care': 'Machine Wash', 'Weave Type': 'Knitted and Woven'}"

In [31]:
df["description"][12123]

"<b>Product design details</b><ul><li>Blue geometric printed sporty jacket,</li><li>stand collar</li><li>2 pockets</li><li>zip closure</li><li>long sleeves</li><li>straight hemline</li><li>mesh lining</li><li>has a mock collar</li><li>Front utility zipper pockets</li><li>Fit: Relaxed</li></ul>Shell: 100% nylon<br>Lining: 100% polyester<br>Machine washThe model (height 6') is wearing a size M"

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14214 entries, 0 to 14213
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   p_id          14214 non-null  int64  
 1   name          14214 non-null  object 
 2   products      14214 non-null  object 
 3   price         14214 non-null  float64
 4   colour        14214 non-null  object 
 5   brand         14214 non-null  object 
 6   img           14214 non-null  object 
 7   description   14214 non-null  object 
 8   p_attributes  14214 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 999.6+ KB
